# Notebook para la tarea relacionada con el tema: Ingenieria de Descriptores

Importacion de paquetes

In [1]:
import sqlite3
import pandas as pd
import datetime as dt

In [2]:
d_ini = dt.datetime(2017,1,1)
d_fin = dt.datetime(2018,1,1)

In [3]:
### importa la informacion de los accidentes
accidentes = pd.read_csv('data/proc_accidentes_2017.csv')
accidentes['TW'] = pd.to_datetime(accidentes['TW'])
accidentes['Accidente'] = 1

In [4]:
### importa la informacion
db_name = "data/clima.sqlite3"
conn = sqlite3.connect(db_name)
query = f"""SELECT *
            FROM clima
            WHERE
            TW >= '{d_ini}' AND
            TW <= '{d_fin}' """

info_clima = pd.read_sql_query(query,conn)
info_clima['TW'] = pd.to_datetime(info_clima['TW'])

to_drop = ['summary','windBearing']
info_clima = info_clima.drop(columns = to_drop)

In [10]:
list(info_clima.columns)

['TW',
 'BARRIO',
 'summary',
 'icon',
 'precipIntensity',
 'precipProbability',
 'temperature',
 'apparentTemperature',
 'dewPoint',
 'humidity',
 'windSpeed',
 'windBearing',
 'cloudCover',
 'uvIndex',
 'visibility']

In [175]:
### Procesamos y guardamos la data procesada
### Completamos la informacion faltante de la serie para cada barrio
### con el promedio de las 10 horas antes y de las proximas 10 horas
database_name = 'data/info_clima.sqlite3'
connection = sqlite3.connect(database_name)

cont = 0
vars = ['precipIntensity','precipProbability']
barrios = info_clima['BARRIO'].unique()
for barrio in barrios:
    cont = cont + 1
    print(f'procesando {cont} de {len(barrios)}')
    info_aux = info_clima[info_clima['BARRIO']==barrio].reset_index(drop = True)
    nan_vars = info_aux.isna().sum()[info_aux.isna().sum()>0].reset_index()['index'].values
    info_aux.index = info_aux.TW
    
    for var in nan_vars:
        nan_times = info_aux['TW'][info_aux[var].isna()].values
        for time in nan_times:
            time = pd.to_datetime(time)
            date_ini = time - dt.timedelta(hours = 10)
            date_fin = time + dt.timedelta(hours = 10)
            
            if var == 'icon':
                aux = info_aux[var][(info_aux['TW']>= date_ini) &(info_aux['TW']<= date_fin)].reset_index().groupby('icon').count().reset_index()
                value = aux['icon'][aux['TW']==aux['TW'].max()].values[0]
            else:    
                value = info_aux[var][(info_aux['TW']>= date_ini) &(info_aux['TW']<= date_fin)].mean()
            
            info_aux.loc[time,var] = value
            
            
    info_aux = info_aux.reset_index(drop = True)
    info_aux.to_sql('clima',connection, if_exists = 'append', index = False)

procesando 1 de 312
procesando 2 de 312
procesando 3 de 312
procesando 4 de 312
procesando 5 de 312
procesando 6 de 312
procesando 7 de 312
procesando 8 de 312
procesando 9 de 312
procesando 10 de 312
procesando 11 de 312
procesando 12 de 312
procesando 13 de 312
procesando 14 de 312
procesando 15 de 312
procesando 16 de 312
procesando 17 de 312
procesando 18 de 312
procesando 19 de 312
procesando 20 de 312
procesando 21 de 312
procesando 22 de 312
procesando 23 de 312
procesando 24 de 312
procesando 25 de 312
procesando 26 de 312
procesando 27 de 312
procesando 28 de 312
procesando 29 de 312
procesando 30 de 312
procesando 31 de 312
procesando 32 de 312
procesando 33 de 312
procesando 34 de 312
procesando 35 de 312
procesando 36 de 312
procesando 37 de 312
procesando 38 de 312
procesando 39 de 312
procesando 40 de 312
procesando 41 de 312
procesando 42 de 312
procesando 43 de 312
procesando 44 de 312
procesando 45 de 312
procesando 46 de 312
procesando 47 de 312
procesando 48 de 312
p

In [176]:
query = f"""SELECT *
            FROM clima
            WHERE
            TW >= '{d_ini}' AND
            TW <= '{d_fin}' """

info_clima2 = pd.read_sql_query(query,connection)
info_clima2['TW'] = pd.to_datetime(info_clima2['TW'])

In [179]:
data = info_clima2.merge(accidentes[['TW','BARRIO','Accidente']], how = 'left', on = ['TW','BARRIO'])
data['Accidente'] = data['Accidente'].fillna(0)

In [180]:
data.shape

(2733120, 14)

In [181]:
info_clima2.shape

(2733120, 13)

In [182]:
data.isna().sum()

TW                     0
BARRIO                 0
icon                   0
precipIntensity        0
precipProbability      0
temperature            0
apparentTemperature    0
dewPoint               0
humidity               0
windSpeed              0
cloudCover             0
uvIndex                0
visibility             0
Accidente              0
dtype: int64

In [183]:
data.ftypes

C:\Users\pasal\Anaconda3\envs\ciencia-datos\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: DataFrame.ftypes is deprecated and will be removed in a future version. Use DataFrame.dtypes instead.
  """Entry point for launching an IPython kernel.


TW                     datetime64[ns]:dense
BARRIO                         object:dense
icon                           object:dense
precipIntensity               float64:dense
precipProbability             float64:dense
temperature                   float64:dense
apparentTemperature           float64:dense
dewPoint                      float64:dense
humidity                      float64:dense
windSpeed                     float64:dense
cloudCover                    float64:dense
uvIndex                       float64:dense
visibility                    float64:dense
Accidente                     float64:dense
dtype: object

In [184]:
100*data['Accidente'].sum()/len(data)

1.496604613042969